In [4]:
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [6]:
class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, dict_op):
        condicion = callable(dict_op) or all((callable(self.operations[key]) for key in self.operations.keys()))
        if condicion:
            self.operations = dict_op
        else:
            raise TypeError("Class input must be either a callable of a dict full of callables")
        #fin if 
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X):
        datos = X.copy()
        eig_pattern = r"^(w\d+|eig\d+|eig_\d+|\(omega\^2\)_\d+)$"
        dim_pattern = r"^(dx|dy|dz|bx|by|bz|lx|ly|lz)$"
        rho_pattern = r"^(rho|Rho|RHO)$"
        eig_columns = list(filter(lambda x: re.match(eig_pattern, x), datos.keys()))
        dim_columns = list(filter(lambda x: re.match(dim_pattern, x), datos.keys()))
        rho_columns = list(filter(lambda x: re.match(rho_pattern, x), datos.keys()))
        cols_transformar = eig_columns + dim_columns + rho_columns
        if callable(self.operations):
            for col in cols_transformar:
                if col in datos.keys():
                    datos[col] = self.operations(datos[col])
                #fin if 
            #fin for
        elif type(self.operations) == dict and all((callable(self.operations[key]) for key in self.operations.keys())): 
            for col in cols_transformar:
                if col in datos.keys() and col in self.operations:
                    datos[col] = self.operations[col](datos[col])
                #fin if 
            #fin for
        else: 
            raise TypeError("Class input must be either a callable of a dict full of callables")
        #fin if 
        return datos
    #fin transform
#fin class